In [5]:
import sys
!{sys.executable} -m pip install pip earthengine-api
!{sys.executable} -m pip install pip geemap
!{sys.executable} -m pip install pip wget

  Using cached earthengine_api-0.1.326-py3-none-any.whl
  Using cached google_api_python_client-1.12.11-py2.py3-none-any.whl (62 kB)
  Using cached httplib2-0.20.4-py3-none-any.whl (96 kB)
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
  Using cached httplib2shim-0.0.3-py2.py3-none-any.whl
  Using cached uritemplate-3.0.1-py2.py3-none-any.whl (15 kB)
  Using cached geemap-0.17.1-py2.py3-none-any.whl (2.1 MB)
  Using cached ee_extra-0.0.14-py3-none-any.whl
  Using cached geeadd-0.5.6-py3-none-any.whl (30 kB)
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using cached geocoder-1.38.1-py2.py3-none-any.whl (98 kB)
  Using cached sankee-0.2.0-py3-none-any.whl
  Using cached logzero-1.7.0-py2.py3-none-any.whl (16 kB)
  Using cached plotly-5.10.0-py2.py3-none-any.whl (15.2 MB)
  Using cached ratelim-0.1.6-py2.py3-none-any.whl (4.0 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=3

In [1]:
import sys
!{sys.executable} -m pip install pip fiona[s3]

In [1]:
import os, requests, json, geojson
import pandas as pd
import geopandas as gpd
#import ee
#import geemap
import fiona
import boto3
from fiona.session import AWSSession
import shapely

In [33]:
aws_creds = json.loads(open('aws_credentials.json').readline())
ACCESS_KEY = aws_creds['ACCESS_KEY']
SECRET_KEY = aws_creds['SECRET_KEY']
SOURCEDATA_BUCKETNAME = 'cities-urbanshift'

In [2]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data"

In [19]:
WDPA_FILENAME = 'data/WDPA_data/WDPA_Oct2022_Public_shp-polygons'
KBA_FILENAME = 'data/KBA_data/KBAsGlobal_2022_March_01_POL'
WDPA_TEMPFILENAME = 'temp_wdpadata'
KBA_TEMPFILENAME = 'temp_kbadata'
OUTPUT_FILENAME = 'LND-7.csv'

In [20]:
s3 = boto3.resource(
    service_name = 's3',
    aws_access_key_id = ACCESS_KEY,
    aws_secret_access_key = SECRET_KEY
)

bucket = s3.Bucket(SOURCEDATA_BUCKETNAME)
for ext in ('shp', 'shx'):
    bucket.download_file('{0}.{1}'.format(WDPA_FILENAME, ext), '{0}.{1}'.format(WDPA_TEMPFILENAME, ext))
    bucket.download_file('{0}.{1}'.format(KBA_FILENAME, ext), '{0}.{1}'.format(KBA_TEMPFILENAME, ext))

1 shp
2 shp
1 shx
2 shx


In [22]:
wdpa_multi = shapely.geometry.MultiPolygon([shapely.geometry.shape(poly['geometry']) for poly in fiona.open('{0}.shp'.format(WDPA_TEMPFILENAME))])
kba_multi = shapely.geometry.MultiPolygon([shapely.geometry.shape(poly['geometry']) for poly in fiona.open('{0}.shp'.format(KBA_TEMPFILENAME))]) 

In [7]:
# get list of urbanshift cities
boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')

# remove cities without tree cover data availability
#tml_not_available_cities = ['BRA-Salvador','MEX-Monterrey']
tml_not_available_cities = []
boundary_georef = boundary_georef[~boundary_georef['geo_name'].isin(tml_not_available_cities)].reset_index(drop=True)
boundary_georef

,geo_name,level,aoi_boundary_name,units_boundary_name,city_name,country_name,country_code,continent
0,ARG-Mendoza,region,ADM3union,ADM3,Mendoza,Argentina,ARG,America
1,ARG-Mar_del_Plata,city,ADM3,ADM4,Mar del Plata city,Argentina,ARG,America
2,ARG-Mar_del_Plata,region,ADM2,NaN,Mar del Plata region,Argentina,ARG,America
3,ARG-Ushuaia,city,ADM4,ADM5,Ushuaia city,Argentina,ARG,America
4,ARG-Ushuaia,region,ADM3,NaN,Ushuaia region,Argentina,ARG,America
5,ARG-Salta,region,ADM2union,ADM3,Salta,Argentina,ARG,America
6,ARG-Buenos_Aires,region,ADM2union,ADM2,Buenos Aires,Argentina,ARG,America
7,BRA-Teresina,city,ADM4union,ADM4,Teresina city,Brazil,BRA,America
8,BRA-Teresina,region,ADM2union,ADM2,Teresina region,Brazil,BRA,America
9,BRA-Florianopolis,city,ADM4union,ADM4,Florianopolis,Brazil,BRA,America


In [8]:
def do_one_geom(row):
    poly = row[0]
    box = poly.bounds
    protected_box = shapely.ops.clip_by_rect(wdpa_multi, *box)
    protected_box = shapely.ops.unary_union(protected_box)    # unary_union repairs inalid geometries
    kba_box = shapely.ops.clip_by_rect(kba_multi, *box)
    kba_box = shapely.ops.unary_union(kba_box)
    key_poly = kba_box.intersection(poly)
    protectedkey_poly = protected_box.intersection(key_poly)
    keypoly_area = key_poly.area
    if keypoly_area > 0:
        result = protectedkey_poly.area/key_poly.area
    else:
        result = -9999
    print('{0}: {1}'.format(row[1], result))
    return result

In [9]:
for i in range(len(boundary_georef)):
    if not OUTPUT_FILENAME in os.listdir('.'):
        so_far_df = pd.DataFrame()
        so_far_df.to_csv(OUTPUT_FILENAME)
        so_far = []
    else:
        so_far_df = pd.read_csv(OUTPUT_FILENAME)
        so_far = [so_far_df.iloc[j]['geo_id'] for j in range(len(so_far_df))]
    
    most_recent = []
    #outputs_df = pd.DataFrame()
    for boundary_name in ['aoi_boundary_name', 'units_boundary_name']:
        if type(boundary_georef.loc[i, boundary_name]) != float: # sometimes boundary_id is nan
            boundary_id = boundary_georef.loc[i, 'geo_name']+'-' + boundary_georef.loc[i, boundary_name]
            boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-' + boundary_id + '.geojson'
            boundary_geo = requests.get(boundary_path).json()
            temp_gdf = gpd.GeoDataFrame.from_features(boundary_geo)
            temp_gdf['LND-7'] = temp_gdf.apply(do_one_geom, axis=1)
            most_recent.append(temp_gdf.copy())
    result = pd.concat([so_far_df] + most_recent, axis=0)
    result[['geometry', 'geo_id', 'geo_level', 'geo_name', 'geo_parent_name', 'creation_date', 'LND-7']].to_csv(OUTPUT_FILENAME)

ARG-Mendoza_ADM-3-union_1: 0.7784686801912738
ARG-Mendoza_ADM-3_1: -9999
ARG-Mendoza_ADM-3_2: -9999
ARG-Mendoza_ADM-3_3: -9999
ARG-Mendoza_ADM-3_4: -9999
ARG-Mendoza_ADM-3_5: -9999
ARG-Mendoza_ADM-3_6: -9999
ARG-Mendoza_ADM-3_7: -9999
ARG-Mendoza_ADM-3_8: -9999
ARG-Mendoza_ADM-3_9: -9999
ARG-Mendoza_ADM-3_10: -9999
ARG-Mendoza_ADM-3_11: -9999
ARG-Mendoza_ADM-3_12: -9999
ARG-Mendoza_ADM-3_13: -9999
ARG-Mendoza_ADM-3_14: -9999
ARG-Mendoza_ADM-3_15: -9999
ARG-Mendoza_ADM-3_16: -9999
ARG-Mendoza_ADM-3_17: -9999
ARG-Mendoza_ADM-3_18: -9999
ARG-Mendoza_ADM-3_19: -9999
ARG-Mendoza_ADM-3_20: -9999
ARG-Mendoza_ADM-3_21: -9999
ARG-Mendoza_ADM-3_22: -9999
ARG-Mendoza_ADM-3_23: -9999
ARG-Mendoza_ADM-3_24: -9999
ARG-Mendoza_ADM-3_25: -9999
ARG-Mendoza_ADM-3_26: -9999
ARG-Mendoza_ADM-3_27: -9999
ARG-Mendoza_ADM-3_28: -9999
ARG-Mendoza_ADM-3_29: -9999
ARG-Mendoza_ADM-3_30: -9999
ARG-Mendoza_ADM-3_31: -9999
ARG-Mendoza_ADM-3_32: -9999
ARG-Mendoza_ADM-3_33: -9999
ARG-Mendoza_ADM-3_34: -9999
ARG-Mendoza

In [26]:
for temp_filename in (WDPA_TEMPFILENAME, KBA_TEMPFILENAME):
    for ext in ('shp', 'shx'):
        os.remove('{0}.{1}'.format(temp_filename, ext))